In [1]:
from IPython.display import display

In [2]:
from dsmlibrary.datanode import DataNode
from tqdm.auto import tqdm
import requests
import pandas as pd

In [3]:
datanode = DataNode()

Init DataNode sucessful!


In [4]:
landing_dir_id = 16
staging_dir_id = 17

### Get Data

#### Board Data

In [5]:
board_data_fID = datanode.get_file_id(directory_id=landing_dir_id ,name="set-board.parquet")
ddf_board_data = datanode.read_df(file_id=board_data_fID)
print(ddf_board_data.shape)
ddf_board_data.head()

(7153, 3)


,name,positions,company
0,นาย ครรชิต สิงห์สุวรรณ์,"ประธานกรรมการ,กรรมการอิสระ,กรรมการตรวจสอบ",2s
1,นาย สมบัติ ลีสวัสดิ์ตระกูล,"ประธานกรรมการบริหาร,กรรมการ",2s
2,นาย สุรพล สติมานนท์,"รองประธานกรรมการ,กรรมการอิสระ,ประธานกรรมการตรว...",2s
3,นาย แสงรุ้ง นิติภาวะชน,"กรรมการผู้จัดการ,กรรมการ",2s
4,น.ส. บุญศรี ปัญญาเปี่ยมศักดิ์,กรรมการ,2s


#### Shareholder Data

In [6]:
shareholder_fID = datanode.get_file_id(directory_id=landing_dir_id ,name="set-shareholder.parquet")
ddf_shareholder = datanode.read_df(file_id=shareholder_fID)
print(ddf_shareholder.shape)
ddf_shareholder.head()

(8180, 7)


,sequence,name,nationality,numberOfShare,percentOfShare,isThaiNVDR,company
0,1,นาย สมบัติ ลีสวัสดิ์ตระกูล,,95234050,19.05,False,2s
1,2,นาย แสงรุ้ง นิติภาวะชน,,29109087,5.82,False,2s
2,3,นาย ปรีดา วงศ์สถิตย์พร,,22397165,4.48,False,2s
3,4,น.ส. ธนิกา ตั้งพูนผลวิวัฒน์,,20048000,4.01,False,2s
4,5,น.ส. นิสาภ์ รุ่งรัตนาอุบล,,17962500,3.59,False,2s


### Merge

In [7]:
ddf_shareholder_board = ddf_board_data.merge(ddf_shareholder, on='name')
print(ddf_shareholder_board.shape)

(2011, 9)


In [8]:
ddf_shareholder_board['positions'] = ddf_shareholder_board['positions'].str.split(',')
ddf_shareholder_board = ddf_shareholder_board.explode('positions')
ddf_shareholder_board = ddf_shareholder_board.rename(columns={'company_x': 'company'})
ddf_shareholder_board = ddf_shareholder_board.drop(columns=['company_y', 'isThaiNVDR'])
ddf_shareholder_board

,name,positions,company,sequence,nationality,numberOfShare,percentOfShare
0,นาย สมบัติ ลีสวัสดิ์ตระกูล,ประธานกรรมการบริหาร,2s,1,,95234050,19.05
0,นาย สมบัติ ลีสวัสดิ์ตระกูล,กรรมการ,2s,1,,95234050,19.05
1,นาย แสงรุ้ง นิติภาวะชน,กรรมการผู้จัดการ,2s,2,,29109087,5.82
1,นาย แสงรุ้ง นิติภาวะชน,กรรมการ,2s,2,,29109087,5.82
2,น.ส. บุญศรี ปัญญาเปี่ยมศักดิ์,กรรมการ,2s,6,,15417277,3.08
...,...,...,...,...,...,...,...
2008,น.ส. จอมขวัญ จิราธิวัฒน์,กรรมการ,zen,4,,34535200,11.51
2009,นาย ศิรุวัฒน์ ชัชวาลย์,กรรมการ,zen,7,,4750000,1.58
2010,นาย ศุภกิจ งามจิตรเจริญ,ประธานกรรมการบริหาร,ziga,1,,118701200,19.37
2010,นาย ศุภกิจ งามจิตรเจริญ,ประธานเจ้าหน้าที่บริหาร,ziga,1,,118701200,19.37


##### Write Data Node (Parquet)

In [9]:
datanode.write(
    df=ddf_shareholder_board, 
    directory=staging_dir_id, 
    name="set-shareholder-board", 
    profiling=True, 
    lineage=[board_data_fID, shareholder_fID]
)

{'sucess': True,
 'file_id': 53,
 'path': 'SothanaV/set-scrapping/staging/set-shareholder-board.parquet',
 'lineage': True}

##### Write CSV

In [10]:
ddf_shareholder_board.to_csv('dsm.tmp/shareholder_board.csv', index=False)

In [11]:
datanode.upload_file(directory_id=staging_dir_id, file_path='dsm.tmp/shareholder_board.csv', description="รายชื่อบุคคลที่เป็นทั้งกรรมการบริษัทและเป็นผู้ถือหุ้นติด 10 อันดับแรกในบริษัทตัวเองจาก SET")

  0%|          | 0.00/358k [00:00<?, ?B/s]

{'id': 54,
 'msg': 'file shareholder_board.csv create sucessful',
 'path': 'SothanaV/set-scrapping/staging/shareholder_board.csv',
 'uploaded': True}